# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#Загружаем нужные нам библиотеки

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
#Функция, которая отобразит полную информацию о файле

def study(df): 
    info = display(df.head(10)), df.info(), display(df.describe())

In [3]:
#Загружаем файл с данными

data = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
#Посмотрим полную информацию о файле

study(data)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Приведем тип данных в столбцах calls и messages к int64. Проверим данные на явные дубликаты.

In [5]:
for column in data[['calls','messages']]:
    data[column] = data[column].astype("int")

In [6]:
data.duplicated().sum()

0

In [7]:
#Посмотрим обновленные данные

study(data)

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0
5,58,344.56,21,15823.37,0
6,57,431.64,20,3738.90,1
7,15,132.40,6,21911.60,0
8,7,43.39,3,2538.67,1
9,90,665.41,38,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


# Вывод

Мы убедились, что данные, в целом, чистые. Привели столбцы calls и messages к типу int. 

## Разбейте данные на выборки

In [8]:
features = data.drop(['is_ultra'], axis=1)

target = data['is_ultra']

Спрятанной тестовой выборки нет. Поэтому, данные разобьем на три части: обучающую, валидационную и тестовую. Исходные данные разбивают в соотношении 3:1:1.

### Обучающая выборка

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.4, 
                                                                            random_state=12345
                                                                           ) 

In [10]:
#Посмотрим размер обучающей выборки

print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


### Валидационная и тестовая выборки

In [11]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, 
                                                                            target_test, 
                                                                            test_size=0.5, 
                                                                            random_state=12345
                                                                           ) 

In [12]:
#Посмотрим размер валидационной выборки

print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


In [13]:
#Посмотрим размер тестовой выборки

print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


## Исследуйте модели

### Decision Tree Classifier

In [14]:
best_model_dtc = None
best_depth_dtc = 0
best_result_dtc = 0
for depth in tqdm(range(1, 11)):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth = depth).fit(features_train, target_train)
    predictions_valid_dtc = model_dtc.predict(features_valid)
    result_dtc = accuracy_score(predictions_valid_dtc, target_valid)
    if best_result_dtc < result_dtc:
        best_model_dtc = model_dtc
        best_depth_dtc = depth
        best_result_dtc = result_dtc

100%|██████████| 10/10 [00:00<00:00, 33.08it/s]


In [15]:
print('best max_depth =', 
      best_depth_dtc, 
      '; ', 
      'best Accuracy = ', 
      best_result_dtc, 
      end =''
     ) 

best max_depth = 3 ;  best Accuracy =  0.7853810264385692

# Вывод

Accuracy лучшей модели из десяти равно 0.7853810264385692, с гиперпараметром max_depth равным - 3, ресурсозатратность же равна, при заданных 10 иттераций, 152.55it/s.

### Random Forest Classifier

In [16]:
best_model_rfc = None
best_est_rfc = 0
best_depth_rfc = 0
best_result_rfc = 0
for est in tqdm(range(10, 51, 10)):
    for depth in range(1, 11):
        model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth).fit(features_train, target_train)
        predictions_valid_rfc = model_rfc.predict(features_valid)
        result_rfc = accuracy_score(predictions_valid_rfc, target_valid)
        if best_result_rfc < result_rfc:
            best_model_rfc = model_rfc
            best_est_rfc = est
            best_depth_rfc = depth
            best_result_rfc = result_rfc

100%|██████████| 5/5 [00:20<00:00,  4.19s/it]


In [17]:
print('best n_estimators =', 
      best_est_rfc, 
      '; ', 
      'best max_depth =', 
      best_depth_rfc, 
      '; ', 
      'best Accuracy = ', 
      best_result_rfc, 
      end =''
     ) 

best n_estimators = 40 ;  best max_depth = 8 ;  best Accuracy =  0.8087091757387247

# Вывод

Accuracy лучшей модели из десяти равно 0.8087091757387247, с гиперпараметром n_estimators = 40 и max_depth = 8, ресурсозатратность же равна, при заданных 10 иттераций, 1.19it/s.

### Logistic Regression

In [18]:
model_lr = LogisticRegression(random_state=12345).fit(features_train, target_train)
predictions_valid_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(predictions_valid_lr, target_valid)
print('Accuracy LogisticRegression:', accuracy_lr, end ='')

Accuracy LogisticRegression: 0.7107309486780715

# Вывод

Accuracy модели Logistic Regression равно 0.7107309486780715.

# Общий вывод

наилучший результат Accuracy у модели решающего дерева(Decision Tree Classifier) - 0.7853810264385692, ресурсозатратность же при этом наименьшая,при заданных 10 иттераций, 152.55it/s;

наилучший результат Accuracy у модели случайного леса(Random Forest Classifier) - 0.80248833592535, ресурсозатратность же, при заданных 10 иттераций, 5.76it/s.

результат Accuracy у модели логистической регрессии(LogisticRegression) - 0.7589424572317263.

Наиболее эффективно (точность и скорость) себя показала модель случайного леса(Random Forest Classifier), поэтому для тестовой проверки выберем её.

## Проверьте модель на тестовой выборке

In [19]:
model_rfc = RandomForestClassifier(random_state=12345, n_estimators = 8, max_depth = 8).fit(features_train, target_train)
predictions_test_rfc = model_rfc.predict(features_test)
result_test_rfc = accuracy_score(predictions_test_rfc, target_test)
print('Accuracy RandomForestClassifier:', result_test_rfc)

Accuracy RandomForestClassifier: 0.7962674961119751


# Вывод

Accuracy у модели случайного леса(Random Forest Classifier) на тестовой выборке равна 0.7962674961119751

## (бонус) Проверьте модели на адекватность

Для проверки используем модель DummyClassifier.

In [20]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_dc)

Accuracy DummyClassifier: 0.7060653188180405


In [21]:
print('Accuracy RandomForestClassifier:', result_test_rfc)

Accuracy RandomForestClassifier: 0.7962674961119751


# Вывод

Accuracy модели DummyClassifier ниже, чем у модели RandomForestClassifier - модель адекватна и эффективна.

# Общий вывод

1) Предоставленные данные оказались чистыми, немного скорректировали типы данных в столбцах calls и messages;

2) Были исследованы модели Decision Tree Classifier, Random Forest Classifier и Logistic Regression;

3) Наилучшую эффективность (точность и скорость) показала модель Random Forest Classifier с гиперпараметрами n_estimators равным - 8 и max_depth равным 8;

4) Модель Random Forest Classifier с наилучшими гиперапараметрами была проверена на тестовой выборке и показала результат Accuracy 0.7962674961119751;

5) Модель Random Forest Classifier, проверенная на тестовой выборке, была проверена на вменяемость: для проверки использовалась модель DummyClassifier, которая показала результат Accuracy - 0.7060653188180405.